In [ ]:
TOKEN="<INPUT_TOKEN>"

In [ ]:
!oc login \
    --token="{TOKEN}" \
    --server=<INPUT_API_URL>

In [ ]:
!oc create secret generic raft-workshop-secrets \
  --from-literal=S3_ENDPOINT="$AWS_S3_ENDPOINT" \
  --from-literal=S3_BUCKET="$AWS_S3_BUCKET" \
  --from-literal=S3_ACCESS_KEY="$AWS_ACCESS_KEY_ID" \
  --from-literal=S3_SECRET_KEY="$AWS_SECRET_ACCESS_KEY"

In [ ]:
%%bash
oc create -f - <<EOF
apiVersion: batch/v1
kind: Job
metadata:
  generateName: granite-ft-surfing-
  labels:
    app: granite-ft-surfing
spec:
  backoffLimit: 0
  template:
    metadata:
      labels:
        app: granite-ft
    spec:
      restartPolicy: Never
      serviceAccountName: default
      containers:
        - name: granite-ft-trainer
          image: docker.io/cengleby86/peft-training-workshop:latest
          imagePullPolicy: Always
          command: ["python"]
          args: ["train.py"]
          envFrom:
            - secretRef:
                name: raft-workshop-secrets
          env:
            - name: S3_NAMESPACE
              valueFrom:
                fieldRef:
                  fieldPath: metadata.namespace
            - name: DATASET_NAME
              value: "surfing"
          resources:
            limits:
              nvidia.com/gpu: "1"
              cpu: "4"
              memory: "20Gi"
            requests:
              cpu: "1"
              memory: "8Gi"
          volumeMounts:
            - name: tmp-volume
              mountPath: /tmp
      volumes:
        - name: tmp-volume
          emptyDir:
            sizeLimit: "40Gi"
      nodeSelector:
        nvidia.com/gpu.present: "true"
EOF